# IMPORT

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import math

# METHOD

### Euclidean Distance(For Numpy Array)

In [8]:
def np_euclidean(e1,e2):
    return math.sqrt(math.fsum(list(map(lambda x: x**2 ,e1-e2))))

### DTW

In [9]:
def dtw_distance(s1, s2):
    DTW={}

    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

### Generate Data and Results for each File

In [10]:
def read_all_lines(file) :
    with open(file, 'rt') as fd:
        convert = [e.split() for e in fd.readlines()]
        all_data = []
        all_results = []
        for line in convert:
            data = line
            result = data.pop(-1)
            all_data.append(data)
            all_results.append(result)
            
        float_data = []
        for element in all_data:
            float_data.append([float(e) for e in element])
        float_data = np.array(float_data)
        all_results = np.array(all_results)
        
        normalized_data = []
        for element in float_data:
            mean = math.fsum(element)/len(element)
            std = math.sqrt(sum((element-mean)**2)/(len(element)-1))
            normalized_data.append((element - mean)/std)
        normalized_data = np.array(normalized_data)
        return normalized_data, all_results

### Calculate F1 Score

In [11]:
def score_eu(test_data,test_results,mrcp_avg,noise_avg):
    e_results = list()
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for each_test_data in test_data:
        e_class = None
        min_dist = float('inf')
        for avg in mrcp_avg:
            dist = np_euclidean(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'MRCP'
        for avg in noise_avg:
            dist = np_euclidean(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'Noise'
        e_results.append(e_class)
    e_results = np.array(e_results)
    for i in range(len(test_data)):
        if e_results[i] == 'MRCP' and test_results[i] == 'MRCP' :
            TP += 1
        elif e_results[i] == 'MRCP' and test_results[i] == 'Noise' :
            FP += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'MRCP' :
            FN += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'Noise' :
            TN += 1
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)*100
    f1_score = (2*precision*recall)/(precision+recall)*100
    return accuracy,f1_score

def score_dtw(test_data,test_results,mrcp_avg,noise_avg):
    e_results = list()
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for each_test_data in test_data:
        e_class = None
        min_dist = float('inf')
        for avg in mrcp_avg:
            dist = dtw_distance(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'MRCP'
        for avg in noise_avg:
            dist = dtw_distance(each_test_data, avg) 
            if dist < min_dist:
                min_dist = dist
                e_class = 'Noise'
        e_results.append(e_class)
    e_results = np.array(e_results)
    for i in range(len(test_data)):
        if e_results[i] == 'MRCP' and test_results[i] == 'MRCP' :
            TP += 1
        elif e_results[i] == 'MRCP' and test_results[i] == 'Noise' :
            FP += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'MRCP' :
            FN += 1
        elif e_results[i] == 'Noise' and test_results[i] == 'Noise' :
            TN += 1
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    accuracy = (TP+TN)/(TP+TN+FP+FN)*100
    f1_score = (2*precision*recall)/(precision+recall)*100
    return accuracy,f1_score

# 1-NN

### Euclidean

In [12]:
print('*----------------------------------------1-NN Euclidean----------------------------------------*')
for i in range(1,10):
    data , results = read_all_lines('training data/participant_'+str(i)+'.txt')
    test_data , test_results = read_all_lines('test data/participant_'+str(i)+'.txt')
    print('Data Participant:',i,data[:0],'\n')
    
    mrcp_data = list()
    noise_data = list()
    mrcp_result = list()
    noise_result = list()

    for each_data, each_result in zip(data,results):
        if each_result == 'MRCP':
            mrcp_data.append(each_data)
            mrcp_result.append(each_result)
        else:
            noise_data.append(each_data)
            noise_result.append(each_result)

    mrcp_data = np.array(mrcp_data)
    noise_data = np.array(noise_data)
    mrcp_results = np.array(mrcp_result)
    noise_results = np.array(noise_result)
    accuracy, f1_score = score_eu(test_data,test_results,mrcp_data,noise_data)
    
    print('F1_score: ',f1_score)
    print('-------------------------------------------------------------------------------------------------')

*----------------------------------------1-NN Euclidean---------------------------------------------------------*
Data Participant: 1 [] 

F1_score:  78.04878048780488
-------------------------------------------------------------------------------------------------
Data Participant: 2 [] 

F1_score:  92.03539823008849
-------------------------------------------------------------------------------------------------
Data Participant: 3 [] 

F1_score:  90.9090909090909
-------------------------------------------------------------------------------------------------
Data Participant: 4 [] 

F1_score:  61.53846153846153
-------------------------------------------------------------------------------------------------
Data Participant: 5 [] 

F1_score:  82.56880733944953
-------------------------------------------------------------------------------------------------
Data Participant: 6 [] 

F1_score:  70.9090909090909
--------------------------------------------------------------------------

### DTW

In [13]:
print('*----------------------------------------1-NN DTW----------------------------------------*')
for i in range(1,10):
    data , results = read_all_lines('training data/participant_'+str(i)+'.txt')
    test_data , test_results = read_all_lines('test data/participant_'+str(i)+'.txt')
    print('Data Participant:',i,data[:0],'\n')
    
    mrcp_data = list()
    noise_data = list()
    mrcp_result = list()
    noise_result = list()

    for each_data, each_result in zip(data,results):
        if each_result == 'MRCP':
            mrcp_data.append(each_data)
            mrcp_result.append(each_result)
        else:
            noise_data.append(each_data)
            noise_result.append(each_result)

    mrcp_data = np.array(mrcp_data)
    noise_data = np.array(noise_data)
    mrcp_results = np.array(mrcp_result)
    noise_results = np.array(noise_result)
    accuracy, f1_score = score_dtw(test_data,test_results,mrcp_data,noise_data)
    
    print('F1_score: ',f1_score)
    print('-------------------------------------------------------------------------------------------------')

*----------------------------------------1-NN DTW---------------------------------------------------------*
Data Participant: 1 [] 

F1_score:  72.58064516129032
-------------------------------------------------------------------------------------------------
Data Participant: 2 [] 

F1_score:  95.79831932773108
-------------------------------------------------------------------------------------------------
Data Participant: 3 [] 

F1_score:  94.21487603305785
-------------------------------------------------------------------------------------------------
Data Participant: 4 [] 

F1_score:  53.57142857142857
-------------------------------------------------------------------------------------------------
Data Participant: 5 [] 

F1_score:  84.21052631578948
-------------------------------------------------------------------------------------------------
Data Participant: 6 [] 

F1_score:  74.28571428571428
------------------------------------------------------------------------------